In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from dateutil.parser import parse

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
#(1.) Function for finding and filling outliers
def outlier_cleaning(df):
  for i in df.columns:   #trying to convert the columns to numbric ;if any can be converted
        try:  
            df[i] = df[i].astype(float)
        except:
            pass  

    #taking only the coloumns which have numerical values
  cols = df._get_numeric_data().columns  
  ab = []
  for i in cols:
    if len(df[i].unique())>(0.20*len(df)):  #Checking if unique values are atleast more than 20% of the values
      ab.append(i)
  df = df.dropna() 
  z_score = np.abs(stats.zscore(df[ab]))   #finding the zscore values; outliers 
  df = df[(z_score < 3).all(axis=1)]       #taking 3 as threshold and removing the outliers and filling some close to mapped values
  return df

In [ ]:
import pandas as pd
from dateutil.parser import parse
#(2.) Date subtraction Function
def format_date(df):
  cols = df.columns
      #initializing a dataframe
  df1 = pd.DataFrame()
      #for loop to iterate the columns 
  for i in cols:
    try:
            #checking if the 1st value of series contains date format
      parse(df[i][0],False)+
           #storing the date format series to df1 dataframe
      df1[i]=df[i]
            #changing the dtype of the series in df1 dataframe to datetime
      df1[i]= pd.to_datetime(df1[i])
            #except blocks to pass the iteration 
    except ValueError:
      pass
    except TypeError:
      pass
          #selecting columns by dtype excluding object type
  df1 = df1.select_dtypes(exclude=['object'])
          #storing the column names of df1
  cols = df1.columns    
  n = len(cols)
  list_1 = []
          #for loop to iterate the number of columns of df1 dataframe
  for i in range(n):
          # append the numbers to list_1
    list_1.append(i)
          # list comprehension for taking two column(numbers) at a time (forming pairs)
  list_2 = [((i), (i + 1) % len(list_1))  for i in range(len(list_1))] 
          #for loop to perform subtraction on 2 columns once
  for i in range(len(list_2)):
          #taking the first values two columns and storing in x,y
    x,y = df1.iloc[0,i],df1.iloc[0,list_2[i][1]]
          # checking which variable value is larger and the performing subtraction
    if x.value>=y.value:
      df1[cols[i]+' - '+cols[list_2[i][1]]] = df1[cols[i]]-df1[cols[list_2[i][1]]]
    else:
      df1[cols[list_2[i][1]]+' - '+cols[i]] = df1[cols[list_2[i][1]]]-df1[cols[i]]
  return df1

In [ ]:
import pandas as pd
# (3.) Function to drop minimum columns having pearson's correlation 
def pearson_test(df):

  ''' Finding Pearson's Correlation for the given dataset and dropping min variables'''
  pearson = df.corr() #initializing pearson corr for df
  cols = pearson.columns 
  for i in range(len(cols)): 
    for j in range(i):
      if pearson.iloc[i,j]>0.85: #if correlation is >85
                                                                #the below condition is for selecting which column to be dropped
        if sum(pearson.nlargest(3,cols[i])[cols[i]])>sum(pearson.nlargest(3,cols[j])[cols[j]]): # comparing the sum of three highest corr values of each column    
          if cols[i] in df.columns:  
            df.drop(columns=cols[i],inplace=True)                       # droping the column which has high corr values compared to corresponding, j column
        elif sum(pearson.nlargest(3,cols[i])[cols[i]])<sum(pearson.nlargest(3,cols[j])[cols[j]]): 
          if cols[j] in df.columns:
            df.drop(columns=cols[j],inplace=True)                       

    return df